<a href="https://colab.research.google.com/github/tayor123/personal-/blob/main/Customer_Behavior_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("customer_behavior.csv")
df.head()


,customer_id,total_rentals,total_amount_spent,avg_rental_rate,first_rental_date,last_rental_date,active_days
0,184,23,90.77,3.946522,26/5/2022 06:55:58+01,23/8/2022 06:56:18+01,89
1,87,30,145.70,4.856667,14/2/2022 16:16:03+01,23/8/2022 16:58:12+01,190
2,477,22,109.78,4.990000,30/5/2022 06:16:06+01,23/8/2022 18:23:30+01,85
3,273,35,157.65,4.504286,25/5/2022 19:46:21+01,23/8/2022 12:50:00+01,89
4,550,32,159.68,4.990000,14/2/2022 16:16:03+01,23/8/2022 20:07:10+01,190


In [ ]:
df['first_rental_date'] = pd.to_datetime(
    df['first_rental_date'],
    dayfirst=True,
    format='mixed'
)

df['last_rental_date'] = pd.to_datetime(
    df['last_rental_date'],
    dayfirst=True,
    format='mixed'
)


In [ ]:
today = df['last_rental_date'].max()

df['days_since_last_rental'] = (today - df['last_rental_date']).dt.days

# Adjusting the threshold to create two classes for 'active_customer'
df['active_customer'] = df['days_since_last_rental'].apply(
    lambda x: 1 if x <= 60 else 0
)


df[['customer_id', 'active_customer']].head()

,customer_id,active_customer
0,184,1
1,87,1
2,477,1
3,273,1
4,550,1


In [ ]:
features = [
    'total_rentals',
    'total_amount_spent',
    'avg_rental_rate',
    'active_days'
]


In [ ]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df['active_customer']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify =y
)

In [ ]:
print("Value counts for 'active_customer' in the training set (y_train):")
display(y_train.value_counts())

print("\nValue counts for 'active_customer' in the test set (y_test):")
display(y_test.value_counts())

Value counts for 'active_customer' in the training set (y_train):


,count
active_customer,
1,479



Value counts for 'active_customer' in the test set (y_test):


,count
active_customer,
1,120


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)

model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [ ]:
print("Value counts for 'active_customer' in the full DataFrame:")
display(df['active_customer'].value_counts())

print("\nValue counts for 'active_customer' in the training set (y_train):")
display(y_train.value_counts())

Value counts for 'active_customer' in the full DataFrame:


,count
active_customer,
1,577
0,22



Value counts for 'active_customer' in the training set (y_train):


,count
active_customer,
1,461
0,18


In [ ]:
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       120

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



In [ ]:
model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.04      0.50      0.08         4
           1       0.97      0.62      0.76       116

    accuracy                           0.62       120
   macro avg       0.51      0.56      0.42       120
weighted avg       0.94      0.62      0.74       120

